In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from keras import layers, Model
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from keras import regularizers, layers, losses
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import precision_recall_curve, confusion_matrix
from keras.utils import plot_model
import pandas as pd
from keras import Input, Model

In [ ]:
# Generate some sample data for the autoencoder
x_train = np.random.rand(1000, 12)
x_test = np.random.rand(100, 12)

In [ ]:
input_dim = 12
latent_dim = 3

class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = Sequential([
            # layers.GaussianNoise(1),
            layers.Dense(8, activation='sigmoid',
                        activity_regularizer=regularizers.l2(10e-3)),
            layers.Dense(4, activation='relu'),
        ])
        self.decoder = Sequential([
            layers.Dense(8, activation='relu'),
            layers.Dense(input_dim, activation='tanh'),
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
autoencoder = Autoencoder(latent_dim)
callback = EarlyStopping(monitor='val_loss', patience=3)
checkpointer = ModelCheckpoint(filepath="DAGMM",
                               monitor='val_loss',
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)

batch_size = 4
nb_epochs = 5
history = autoencoder.fit(x_train, x_train,
                        epochs=nb_epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(x_test, x_test),
                        callbacks=[checkpointer, tensorboard]).history

autoencoder.summary()

plt.plot(range(nb_epochs), history['loss'], label='Training Loss')
plt.plot(range(nb_epochs), history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Use the trained autoencoder to encode the training data
encoded_data = autoencoder.encoder(x_train).numpy()

# Fit a Gaussian Mixture Model with 2 components to the encoded training data
gmm = GaussianMixture(n_components=2)
gmm.fit(encoded_data)

# Use the GMM to predict the probability densities for the encoded test data
encoded_test_data = autoencoder.encoder(x_test).numpy()
scores = gmm.score_samples(encoded_test_data)

# Identify anomalies based on the GMM probability densities
threshold = np.percentile(scores, 5)
anomalies = x_test[scores < threshold]

In [ ]:
# Calculate precision, recall, and F1 score
y_true = np.zeros_like(scores)
y_true[scores < threshold] = 1
y_pred = np.zeros_like(scores)
y_pred[scores < threshold] = 1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")

# Generate precision-recall curve with F1 score
precision, recall, thresholds = precision_recall_curve(y_true, scores)
f1_scores = 2 * precision * recall / (precision + recall)
plt.plot(recall, precision, label='Precision-Recall curve')
plt.plot(recall, f1_scores, label='F1 score')
plt.xlabel('Recall')
plt.ylabel('Precision/F1 score')
plt.legend()
plt.show()

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Generate score with threshold graph
plt.plot(thresholds, precision[:-1], label='Precision')
plt.plot(thresholds, recall[:-1], label='Recall')
plt.xlabel('Threshold')
plt.legend()
plt.show()
